### Final Project SBA Loan Data
### DSC 630
### Sam Loyd
### November 2020
### Pycaret_Cat

### This code was used to select the best classifiction model for loan repayment.

In [1]:
# Required libraries
import pandas as pd
import numpy as np
# import matplotlib.pyplot as pt
import sklearn
from scipy.stats import pearsonr
import matplotlib.pyplot as matplotlib
from matplotlib.lines import Line2D
from matplotlib import pyplot
pd.options.display.float_format = '{:.2f}'.format
import seaborn 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from IPython.display import Image
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
# from pycaret.regression import *
from pycaret.classification import *

In [2]:
raw_sba_df = pd.read_csv("C:\\Users\\Loyd\\Documents\\630Project\\sbapython.csv")

raw_sba_df = raw_sba_df.drop(columns=['ApprovalFY','Month_Jan','Month_Feb','Month_Mar','Month_Apr','Month_May','Month_Jun','Month_Jul','Month_Aug','Month_Sep','Month_Oct','Month_Nov','Month_Dec'])
raw_sba_df

,Term,NoEmp,CreateJob,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,UrbanRural_0,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
0,84,4,0,60000,48000,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,60,2,0,40000,32000,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,180,7,0,287000,215250,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,1
3,60,2,0,35000,28000,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,240,14,7,229000,229000,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897105,60,6,0,70000,56000,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,1
897106,60,6,0,85000,42500,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,1
897107,108,26,0,300000,225000,0,1,0,0,1,...,0,0,0,0,1,1,1,1,0,1
897108,60,6,0,75000,60000,0,1,0,0,1,...,0,0,1,1,1,1,0,0,0,1


#### Normalize the data in case some models require it.  Non-gaussian so normalizing instead of standardizing.

In [3]:
# Normalize the data since its non-Gaussian
# scaler = MinMaxScaler() 
# sba_df[['Term','NoEmp','']] = scaler.fit_transform(data)
sba_df = raw_sba_df.copy()
sba_df[['Term','NoEmp','GrAppv','SBA_Appv']] = minmax_scale(sba_df[['Term','NoEmp','GrAppv','SBA_Appv']])

sba_df

,Term,NoEmp,CreateJob,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,UrbanRural_0,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
0,0.15,0.00,0,0.01,0.01,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0.11,0.00,0,0.01,0.01,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0.32,0.00,0,0.06,0.06,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,1
3,0.11,0.00,0,0.01,0.01,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0.42,0.00,7,0.05,0.06,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897105,0.11,0.00,0,0.01,0.01,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,1
897106,0.11,0.00,0,0.02,0.01,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,1
897107,0.19,0.00,0,0.06,0.06,0,1,0,0,1,...,0,0,0,0,1,1,1,1,0,1
897108,0.11,0.00,0,0.01,0.02,0,1,0,0,1,...,0,0,1,1,1,1,0,0,0,1


#### Split the data into 60,20,20 Train, Test Validate Sets.

In [4]:
# Create X from the features
X = sba_df.drop(columns=['MIS_logical','CreateJob'])

# Create y from output
y = sba_df[['MIS_logical','CreateJob']]
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [5]:
y_train

,MIS_logical,CreateJob
623129,1,0
589937,1,4
576584,1,0
618776,1,0
466287,1,0
...,...,...
111316,1,0
883883,1,14
374049,1,0
54901,0,0


In [6]:
X_train

,Term,NoEmp,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,UrbanRural_0,UrbanRural_1,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
623129,0.11,0.00,0.02,0.02,0,1,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
589937,0.42,0.00,0.04,0.04,0,1,0,0,0,1,...,0,0,1,1,0,1,1,1,1,0
576584,0.49,0.00,0.21,0.21,0,1,0,0,0,1,...,0,0,1,0,1,0,0,1,1,0
618776,0.15,0.00,0.05,0.03,0,1,0,0,0,1,...,0,0,0,1,1,0,1,0,0,0
466287,0.21,0.00,0.05,0.06,0,1,0,0,1,0,...,1,0,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111316,0.21,0.00,0.15,0.15,0,0,1,0,1,0,...,0,1,1,1,1,1,0,0,0,1
883883,0.42,0.00,0.07,0.09,0,0,1,0,0,1,...,0,1,1,1,0,0,0,1,0,0
374049,0.14,0.00,0.01,0.01,0,1,0,0,0,1,...,0,0,1,1,0,1,1,1,1,0
54901,0.07,0.00,0.02,0.01,0,1,0,0,0,1,...,0,0,0,0,0,0,1,1,1,0


In [7]:
# Now try with classification models for MIS_logical
X_train_caret_cat = X_train.copy()
X_train_caret_cat['MIS_logical'] = y_train['MIS_logical'].astype('bool')
X_train_caret_cat

,Term,NoEmp,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,UrbanRural_0,UrbanRural_1,...,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13,MIS_logical
623129,0.11,0.00,0.02,0.02,0,1,0,0,1,0,...,0,0,0,0,0,1,1,0,0,True
589937,0.42,0.00,0.04,0.04,0,1,0,0,0,1,...,0,1,1,0,1,1,1,1,0,True
576584,0.49,0.00,0.21,0.21,0,1,0,0,0,1,...,0,1,0,1,0,0,1,1,0,True
618776,0.15,0.00,0.05,0.03,0,1,0,0,0,1,...,0,0,1,1,0,1,0,0,0,True
466287,0.21,0.00,0.05,0.06,0,1,0,0,1,0,...,0,1,1,1,1,1,1,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111316,0.21,0.00,0.15,0.15,0,0,1,0,1,0,...,1,1,1,1,1,0,0,0,1,True
883883,0.42,0.00,0.07,0.09,0,0,1,0,0,1,...,1,1,1,0,0,0,1,0,0,True
374049,0.14,0.00,0.01,0.01,0,1,0,0,0,1,...,0,1,1,0,1,1,1,1,0,True
54901,0.07,0.00,0.02,0.01,0,1,0,0,0,1,...,0,0,0,0,0,1,1,1,0,False


In [8]:
exp_clf_cat = setup(X_train_caret_cat, target='MIS_logical')

Setup Succesfully Completed!


,Description,Value
0,session_id,4720
1,Target Type,Binary
2,Label Encoded,"False: 0, True: 1"
3,Original Data,"(538266, 56)"
4,Missing Values,False
5,Numeric Features,4
6,Categorical Features,51
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [9]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors.KNeighborsClassifier,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model.SGDClassifier,True
rbfsvm,SVM - Radial Kernel,sklearn.svm.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process.GPC,False
mlp,MLP Classifier,sklearn.neural_network.MLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model.RidgeClassifier,True


In [10]:
compare_models(exclude = ['svm'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.9454,0.9769,0.9728,0.9614,0.9671,0.8075,0.8080,65.2207
1,Extreme Gradient Boosting,0.9437,0.9760,0.9718,0.9604,0.9660,0.8015,0.8020,35.0185
2,Light Gradient Boosting Machine,0.9399,0.9734,0.9724,0.9554,0.9638,0.7856,0.7867,3.1388
3,Gradient Boosting Classifier,0.9246,0.9580,0.9721,0.9386,0.9551,0.7217,0.7260,117.7144
4,Random Forest Classifier,0.9202,0.9241,0.9676,0.9376,0.9524,0.7077,0.7110,3.9060
5,Decision Tree Classifier,0.9118,0.8507,0.9450,0.9478,0.9464,0.6975,0.6975,7.9694
6,Ada Boost Classifier,0.9015,0.9360,0.9577,0.9254,0.9413,0.6369,0.6407,30.5169
7,Extra Trees Classifier,0.8971,0.9071,0.9716,0.9097,0.9396,0.5944,0.6095,48.8056
8,K Neighbors Classifier,0.8739,0.8320,0.9563,0.8974,0.9259,0.5053,0.5171,188.3537
9,Logistic Regression,0.8538,0.8432,0.9744,0.8652,0.9166,0.3429,0.3879,7.1251
